<a href="https://colab.research.google.com/github/uanak/DXbootcamp/blob/main/bootCamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

各種インストール
mecab(要素解析）,pytorch,kmeans（クラスタ）

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install -U torch torchvision
!pip install kmeans-pytorch

import MeCab
import numpy as np
import torch
from kmeans_pytorch import kmeans
from gensim import corpora, matutils
from google.colab import drive
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader
drive.mount('/content/drive')


device = torch.device('cuda:0')


In [ ]:
mecab = MeCab.Tagger("-Ochasen")
mecab.parse('')

def bunkai(contents):
    ret = []
    for  content in contents:
        ret.append(bunkaiChi(content))
    return ret

def bunkaiChi(content):
  return [token for token in doMecab(content)]

def doMecab(text):
  nodes = mecab.parseToNode(text)
  while nodes:
    if nodes.feature.split(',')[0] == '名詞':
      yield nodes.surface.lower()
    nodes = nodes.next


test1 = ["トランプ氏、2州で集会強行　コロナ拡大地域、熱気演出",
         "東京都 新型コロナ 2人が死亡 新たに132人感染確認",
         "iPhoneやApple Watchの「Suica」と「PASMO」って何が違う？　比較してみよう【2020年最新版"]
wordlists = bunkai(test1)

# wordlists=bunkai()
print(wordlists)

jisho = corpora.Dictionary(wordlists)
print(jisho.token2id)
courpus = [jisho.doc2bow(word) for word in wordlists]
print(courpus)

def toVec(wordlist):
  vecs=[]
  for word in wordlist:
    vecs.append(jisho.doc2bow(word))
  return vecs 

data = torch.tensor(toVec(wordlists))
# print(data)

# dense = list(matutils.corpus2dense([vec], num_terms=len(jisho)).T[0])
# print(dense)

[['トランプ', '氏', '2', '州', '集会', '強行', 'コロナ', '拡大', '地域', '熱気', '演出'], ['東京', '都', '新型', 'コロナ', '2', '人', '死亡', '新た', '132', '人', '感染', '確認'], ['iphone', 'apple', 'watch', 'suica', 'pasmo', '何', '比較', '2020', '年', '最新', '版']]
{'2': 0, 'コロナ': 1, 'トランプ': 2, '地域': 3, '州': 4, '強行': 5, '拡大': 6, '氏': 7, '演出': 8, '熱気': 9, '集会': 10, '132': 11, '人': 12, '感染': 13, '新た': 14, '新型': 15, '東京': 16, '死亡': 17, '確認': 18, '都': 19, '2020': 20, 'apple': 21, 'iphone': 22, 'pasmo': 23, 'suica': 24, 'watch': 25, '何': 26, '年': 27, '最新': 28, '比較': 29, '版': 30}
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(0, 1), (1, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)], [(20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]]
tensor([[[ 0,  1],
         [ 1,  1],
         [ 2,  1],
         [ 3,  1],
         [ 4,  1],
         [ 5,  1],
         [ 6,  1],
         [ 7,  1],
         [ 8,  1]

In [ ]:
class LSTMClassifier(nn.Module):
    # モデルで使う各ネットワークをコンストラクタで定義
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。
        self.hidden_dim = hidden_dim
        # インプットの単語をベクトル化するために使う
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # LSTMの隠れ層。これ１つでOK。超便利。
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        self.softmax = nn.LogSoftmax(dim=1)

    # 順伝播処理はforward関数に記載
    def forward(self, sentence):
        # 文章内の各単語をベクトル化して出力。2次元のテンソル
        embeds = self.word_embeddings(sentence)
        # 2次元テンソルをLSTMに食わせられる様にviewで３次元テンソルにした上でLSTMへ流す。
        # 上記で説明した様にmany to oneのタスクを解きたいので、第二戻り値だけ使う。
        _, lstm_out = self.lstm(embeds.view(len(sentence), 1, -1))
        # lstm_out[0]は３次元テンソルになってしまっているので2次元に調整して全結合。
        tag_space = self.hidden2tag(lstm_out[0].view(-1, HIDDEN_DIM))
        # softmaxに食わせて、確率として表現
        tag_scores = self.softmax(tag_space)
        return tag_scores

In [ ]:

dataframe= pd.read_csv('drive/My Drive/dxboot/test.CSV')

x=[]
y=[]
for dataframe:
  i=1
  dataframe.iloc[1,1]

print(data)
# DataLoaderの設定
batch_size = 50
train_loader = DataLoader(data, 
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(data,
                         batch_size=batch_size,
                         shuffle=False)


datasets = pd.DataFrame(columns=["title", "category"])

datasets.head()
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 1024)  # 全結合層
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 1)  # バッチサイズ×入力の数
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
      D                                       マスクで顔を隠す独女たち
0     D                        薄っぺらい親友関係が増殖中？ 女の友情はきょわい〜！？
1     D             【オトナ女子映画部】“隙だらけ”のウザい女に学ぶ男心のつかみ方『乱暴と待機』
2     D  女医が教えるオンナの体のウソホント vol.10「体の変化とホルモン分泌の周期」presen...
3     D                          結婚相手に妥協できるもの、できないもの　（男性編）
4     D  女医が教えるオンナの体のウソホント vol.9「体温上げて、熱いオンナになるべし！」pres...
...  ..                                                ...
4267  K                             「2012年期待の女優ランキング」に非難続出
4268  K                                   ドリムス出演のMステ視聴率が話題
4269  K                          被災者イジメ「津波で死ねばよかったのに」に非難殺到
4270  K                          バットの上に長時間正座…プロ野球界の主従関係を非難
4271  K                              オセロ中島に対する松嶋のコメントに批判の声

[4272 rows x 2 columns]
Net(
  (fc1): Linear(in_features=1, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): 

In [ ]:
class news(Dataset):
    def __init__(self, csv_path):
        # csv ファイルを読み込む。
        df = pd.read_csv(csv_path)
        data = df.iloc[:, 1:]  # データ (2 ~ 14列目)
        labels = df.iloc[:, 0]  # ラベル (1列目)
        # データを標準化する。
        data = normalize(data)
        # クラス ID を 0 始まりにする。[1, 2, 3] -> [0, 1, 2]
        labels -= 1

        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        """サンプルを返す。
        """
        return self.data[index], self.labels[index]

    def __len__(self):
        """csv の行数を返す。
        """
        return len(self.data)

In [ ]:
# data
data_size, dims, num_clusters = 1000, 2, 3
x = np.random.randn(data_size, dims) / 6
x = torch.from_numpy(x)



s = pd.Series([title, cat], index=datasets.columns)
datasets = datasets.append(s, ignore_index=True)
datasets = datasets.sample(frac=1).reset_index(drop=True)
datasets.head()